<a href="https://colab.research.google.com/github/XuanGiaHanNguyen/AfterLife/blob/main/PubMedBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install transformers==4.40.0
!pip3 install datasets
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip3 install seqeval          # for NER evaluation (entity-level F1)
!pip3 install scikit-learn
!pip3 install accelerate
!pip3 install numpy pandas tqdm

Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:
import torch
print(torch.cuda.is_available())       # Should print True
print(torch.cuda.get_device_name(0))   # Shows your GPU name

True
Tesla T4


In [ ]:
from transformers import AutoTokenizer, AutoModel

model_name = "microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

print("Model loaded successfully!")
print(f"Parameters: {sum(p.numel() for p in model.parameters()):,}")
# Should print ~110 million parameters

/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:949: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Model loaded successfully!
Parameters: 109,482,240


**NER test:**



Tried many data-set download, but failed so have to use kaggle

In [ ]:
import kagglehub

# Download JNLPBA - actual BLURB benchmark dataset
path = kagglehub.dataset_download("athibant/jnlpba")
print("Path:", path)

100%|██████████| 1.36M/1.36M [00:01<00:00, 1.34MB/s]

Extracting files...
Path: /root/.cache/kagglehub/datasets/athibant/jnlpba/versions/2


In [ ]:
import os
for f in os.listdir(path):
    print(f)

JNLPBA


See in the files

In [ ]:
import os
for f in os.listdir(path):
    print(f)

JNLPBA


In [ ]:
for root, dirs, files in os.walk(path):
    for f in files:
        print(os.path.join(root, f))

/root/.cache/kagglehub/datasets/athibant/jnlpba/versions/2/JNLPBA/devel.tsv
/root/.cache/kagglehub/datasets/athibant/jnlpba/versions/2/JNLPBA/test.tsv
/root/.cache/kagglehub/datasets/athibant/jnlpba/versions/2/JNLPBA/train.tsv
/root/.cache/kagglehub/datasets/athibant/jnlpba/versions/2/JNLPBA/classes.txt


In [ ]:
Load and inspect data

In [ ]:
def load_conll(filepath):
    sentences, labels = [], []
    tokens, tags = [], []
    with open(filepath) as f:
        for line in f:
            line = line.strip()
            if line == "":
                if tokens:
                    sentences.append(tokens)
                    labels.append(tags)
                    tokens, tags = [], []
            else:
                parts = line.split("\t")
                tokens.append(parts[0])
                tags.append(parts[-1])
    return sentences, labels

base = "/root/.cache/kagglehub/datasets/athibant/jnlpba/versions/2/JNLPBA"

train_sents, train_labels = load_conll(f"{base}/train.tsv")
dev_sents,   dev_labels   = load_conll(f"{base}/devel.tsv")
test_sents,  test_labels  = load_conll(f"{base}/test.tsv")

print(f"Train: {len(train_sents)} sentences")
print(f"Dev:   {len(dev_sents)} sentences")
print(f"Test:  {len(test_sents)} sentences")

Train: 18607 sentences
Dev:   1939 sentences
Test:  4260 sentences


Preview and check lable

In [ ]:
print("Sample tokens:", train_sents[0])
print("Sample labels:", train_labels[0])

unique_labels = sorted(set(l for labels in train_labels for l in labels))
print("\nUnique labels:", unique_labels)
print("Total label types:", len(unique_labels))

Sample tokens: ['-DOCSTART-']
Sample labels: ['O']

Unique labels: ['B-DNA', 'B-RNA', 'B-cell_line', 'B-cell_type', 'B-protein', 'I-DNA', 'I-RNA', 'I-cell_line', 'I-cell_type', 'I-protein', 'O']
Total label types: 11


Building full pipeline

In [ ]:
unique_labels = ['B-DNA', 'B-RNA', 'B-cell_line', 'B-cell_type', 'B-protein',
                 'I-DNA', 'I-RNA', 'I-cell_line', 'I-cell_type', 'I-protein', 'O']

label2id = {l: i for i, l in enumerate(unique_labels)}
id2label = {i: l for i, l in enumerate(unique_labels)}

print("Label mappings ready:", label2id)

Label mappings ready: {'B-DNA': 0, 'B-RNA': 1, 'B-cell_line': 2, 'B-cell_type': 3, 'B-protein': 4, 'I-DNA': 5, 'I-RNA': 6, 'I-cell_line': 7, 'I-cell_type': 8, 'I-protein': 9, 'O': 10}


Building dataset class

In [ ]:
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract")

class NERDataset(Dataset):
    def __init__(self, sentences, labels, tokenizer, max_len=128):
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        tokens = self.sentences[idx]
        labels = self.labels[idx]

        encoding = self.tokenizer(
            tokens,
            is_split_into_words=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        # Align labels to subword tokens
        word_ids = encoding.word_ids()
        aligned_labels = []
        prev_word_id = None
        for word_id in word_ids:
            if word_id is None:
                aligned_labels.append(-100)  # special tokens → ignore
            elif word_id != prev_word_id:
                aligned_labels.append(label2id[labels[word_id]])
            else:
                aligned_labels.append(-100)  # subword continuation → ignore
            prev_word_id = word_id

        return {
            'input_ids':      encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'token_type_ids': encoding['token_type_ids'].squeeze(),
            'labels':         torch.tensor(aligned_labels)
        }

# Filter out -DOCSTART- sentences
def filter_docstart(sents, labs):
    return zip(*[(s, l) for s, l in zip(sents, labs) if s != ['-DOCSTART-']])

tr_s, tr_l = filter_docstart(train_sents, train_labels)
dv_s, dv_l = filter_docstart(dev_sents,   dev_labels)
te_s, te_l = filter_docstart(test_sents,  test_labels)

train_dataset = NERDataset(list(tr_s), list(tr_l), tokenizer)
dev_dataset   = NERDataset(list(dv_s), list(dv_l), tokenizer)
test_dataset  = NERDataset(list(te_s), list(te_l), tokenizer)

print(f"Train: {len(train_dataset)} | Dev: {len(dev_dataset)} | Test: {len(test_dataset)}")
print("Sample input_ids shape:", train_dataset[0]['input_ids'].shape)
print("Sample labels shape:",    train_dataset[0]['labels'].shape)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:949: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Train: 16807 | Dev: 1739 | Test: 3856
Sample input_ids shape: torch.Size([128])
Sample labels shape: torch.Size([128])


Loading the model

In [ ]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    "microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract",
    num_labels=len(unique_labels),
    id2label=id2label,
    label2id=label2id
)
model = model.cuda()
print("Model loaded on GPU")
print(f"Output labels: {model.config.num_labels}")

Some weights of BertForTokenClassification were not initialized from the model checkpoint at microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded on GPU
Output labels: 11


Training loop

In [ ]:
from torch.utils.data import DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup
from seqeval.metrics import f1_score, classification_report
import numpy as np
from tqdm import tqdm

# Hyperparameters (matching the paper)
EPOCHS     = 5
LR         = 3e-5
BATCH_SIZE = 32

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
dev_loader   = DataLoader(dev_dataset,   batch_size=BATCH_SIZE)
test_loader  = DataLoader(test_dataset,  batch_size=BATCH_SIZE)

optimizer = AdamW(model.parameters(), lr=LR, weight_decay=0.01)
total_steps = len(train_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(0.1 * total_steps),
    num_training_steps=total_steps
)

def evaluate(model, loader):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for batch in loader:
            input_ids      = batch['input_ids'].cuda()
            attention_mask = batch['attention_mask'].cuda()
            token_type_ids = batch['token_type_ids'].cuda()
            labels         = batch['labels'].cuda()

            outputs = model(input_ids=input_ids,
                           attention_mask=attention_mask,
                           token_type_ids=token_type_ids)

            preds = torch.argmax(outputs.logits, dim=-1)

            for pred_seq, label_seq in zip(preds, labels):
                pred_list, label_list = [], []
                for p, l in zip(pred_seq, label_seq):
                    if l.item() == -100:
                        continue
                    pred_list.append(id2label[p.item()])
                    label_list.append(id2label[l.item()])
                all_preds.append(pred_list)
                all_labels.append(label_list)

    f1 = f1_score(all_labels, all_preds)
    return f1, all_preds, all_labels

# Training
best_dev_f1  = 0
best_epoch   = 0

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        input_ids      = batch['input_ids'].cuda()
        attention_mask = batch['attention_mask'].cuda()
        token_type_ids = batch['token_type_ids'].cuda()
        labels         = batch['labels'].cuda()

        outputs = model(input_ids=input_ids,
                       attention_mask=attention_mask,
                       token_type_ids=token_type_ids,
                       labels=labels)

        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    avg_loss = total_loss / len(train_loader)
    dev_f1, _, _ = evaluate(model, dev_loader)

    print(f"Epoch {epoch+1} | Loss: {avg_loss:.4f} | Dev F1: {dev_f1:.4f}")

    if dev_f1 > best_dev_f1:
        best_dev_f1 = dev_f1
        best_epoch  = epoch + 1
        torch.save(model.state_dict(), "best_model.pt")
        print(f"  ✅ New best saved!")

print(f"\nBest Dev F1: {best_dev_f1:.4f} at epoch {best_epoch}")

/usr/local/lib/python3.12/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1/5: 100%|██████████| 526/526 [05:48<00:00,  1.51it/s]


Epoch 1 | Loss: 0.4145 | Dev F1: 0.7748
  ✅ New best saved!


Epoch 2/5: 100%|██████████| 526/526 [05:53<00:00,  1.49it/s]


Epoch 2 | Loss: 0.1382 | Dev F1: 0.7864
  ✅ New best saved!


Epoch 3/5: 100%|██████████| 526/526 [05:54<00:00,  1.49it/s]


Epoch 3 | Loss: 0.1106 | Dev F1: 0.7864
  ✅ New best saved!


Epoch 4/5: 100%|██████████| 526/526 [05:53<00:00,  1.49it/s]


Epoch 4 | Loss: 0.0898 | Dev F1: 0.7896
  ✅ New best saved!


Epoch 5/5: 100%|██████████| 526/526 [05:54<00:00,  1.49it/s]


Epoch 5 | Loss: 0.0758 | Dev F1: 0.7907
  ✅ New best saved!

Best Dev F1: 0.7907 at epoch 5


In [ ]:
from seqeval.metrics import classification_report

# Load best model
model.load_state_dict(torch.load("best_model.pt"))

test_f1, test_preds, test_labels_list = evaluate(model, test_loader)
print(f"Test F1: {test_f1:.4f}")
print(f"Paper target: 0.7910")
print(f"Gap: {test_f1 - 0.7910:+.4f}")
print()
print(classification_report(test_labels_list, test_preds))

Test F1: 0.7488
Paper target: 0.7910
Gap: -0.0422

              precision    recall  f1-score   support

         DNA       0.70      0.76      0.73      1054
         RNA       0.70      0.76      0.73       118
   cell_line       0.52      0.72      0.61       500
   cell_type       0.77      0.73      0.75      1920
     protein       0.71      0.84      0.77      5064

   micro avg       0.70      0.80      0.75      8656
   macro avg       0.68      0.76      0.72      8656
weighted avg       0.71      0.80      0.75      8656



**PICO Extraction**

Downloading EBM-NLP from Github


In [ ]:
!git clone https://github.com/bepnye/EBM-NLP.git
!ls EBM-NLP/

Cloning into 'EBM-NLP'...
remote: Enumerating objects: 200, done.
remote: Total 200 (delta 0), reused 0 (delta 0), pack-reused 200 (from 1)
Receiving objects: 100% (200/200), 83.47 MiB | 11.43 MiB/s, done.
Resolving deltas: 100% (82/82), done.
ebm_nlp_1_00.tar.gz  ebm_nlp_difficulty.tar.gz	phase1_example.png
ebm_nlp_2_00.tar.gz  generate_bio_labels.py	README.md
ebm_nlp_demo.py      models			README.txt


In [ ]:
for root, dirs, files in os.walk("EBM-NLP"):
    depth = root.replace("EBM-NLP", '').count(os.sep)
    if depth < 4:
        indent = '  ' * depth
        print(f"{indent}{os.path.basename(root)}/")
        for f in files[:3]:
            print(f"{indent}  {f}")

EBM-NLP/
  .gitattributes
  ebm_nlp_difficulty.tar.gz
  ebm_nlp_1_00.tar.gz
  .git/
    HEAD
    description
    index
    hooks/
      pre-push.sample
      pre-receive.sample
      prepare-commit-msg.sample
    info/
      exclude
    logs/
      HEAD
      refs/
    objects/
      info/
      pack/
        pack-3b810602da3c8d1e11e6fb0a35b78e76f36a3dae.pack
        pack-3b810602da3c8d1e11e6fb0a35b78e76f36a3dae.idx
    branches/
    refs/
      heads/
        master
      remotes/
      tags/
  models/
    eval.py
    logreg/
      logreg.py
    lstm-crf/
      LICENSE.txt
      requirements.txt
      makefile
      model/
        config.py
        base_model.py
        general_utils.py
    crf/
      p1/
        cr-v.sh
        model.py
      p2/
        crf.py


The actual data is in the tar.gz files. Let's extract them:

In [ ]:
!tar -xzf EBM-NLP/ebm_nlp_1_00.tar.gz -C EBM-NLP/
!ls EBM-NLP/

ebm_nlp_1_00	     ebm_nlp_demo.py		models		    README.txt
ebm_nlp_1_00.tar.gz  ebm_nlp_difficulty.tar.gz	phase1_example.png
ebm_nlp_2_00.tar.gz  generate_bio_labels.py	README.md


In [ ]:
for root, dirs, files in os.walk("EBM-NLP/ebm_nlp_1_00"):
    depth = root.replace("EBM-NLP/ebm_nlp_1_00", '').count(os.sep)
    if depth < 4:
        indent = '  ' * depth
        print(f"{indent}{os.path.basename(root)}/")
        for f in files[:3]:
            print(f"{indent}  {f}")

ebm_nlp_1_00/
  ._.DS_Store
  README.txt
  .DS_Store
  annotations/
    ._.DS_Store
    .DS_Store
    aggregated/
      ._.DS_Store
      .DS_Store
      starting_spans/
        ._.DS_Store
        .DS_Store
      hierarchical_labels/
    individual/
      ._.DS_Store
      .DS_Store
      starting_spans/
        ._.DS_Store
        .DS_Store
      hierarchical_labels/
  documents/
    25313065.tokens
    12486433.tokens
    22972771.tokens


In [ ]:
import os
base2 = "EBM-NLP/ebm_nlp_1_00"

# Check what's in aggregated/starting_spans
ann_path = f"{base2}/annotations/aggregated/starting_spans"
for item in os.listdir(ann_path):
    print(item)

._.DS_Store
outcomes
interventions
participants
.DS_Store


In [ ]:
# See subfolders
for folder in os.listdir(ann_path):
    subpath = f"{ann_path}/{folder}"
    if os.path.isdir(subpath):
        files = os.listdir(subpath)
        print(f"{folder}/: {files[:3]}")

# Read one sample annotation
import glob
sample_ann = glob.glob(f"{ann_path}/**/*.ann", recursive=True)[0]
print(f"\nFile: {sample_ann}")
with open(sample_ann) as f:
    print(f.read()[:300])

# Read matching token file
doc_id = os.path.basename(sample_ann).replace(".ann", "")
with open(f"{base2}/documents/{doc_id}.tokens") as f:
    print(f"\nTokens:\n{f.read()[:300]}")

outcomes/: ['train', 'test']
interventions/: ['._.DS_Store', 'train', '.DS_Store']
participants/: ['train', 'test']

File: EBM-NLP/ebm_nlp_1_00/annotations/aggregated/starting_spans/outcomes/train/26449739_AGGREGATED.ann
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,


FileNotFoundError: [Errno 2] No such file or directory: 'EBM-NLP/ebm_nlp_1_00/documents/26449739_AGGREGATED.tokens'

The annotation format is clear — it's a comma-separated list of 0s and 1s, one per token. The filename just needs the _AGGREGATED stripped. Let's fix and build the full loader:

In [ ]:
label_map = {"participants": "P", "interventions": "I", "outcomes": "O"}

# Check first few docs
count = 0
for doc_id in sorted(os.listdir(f"{ann_base}/participants/train"))[:5]:
    doc_id = doc_id.replace("_AGGREGATED.ann", "")
    tok_file = f"{doc_base}/{doc_id}.tokens"

    if not os.path.exists(tok_file):
        print(f"{doc_id}: token file missing")
        continue

    with open(tok_file) as f:
        tokens = [t.strip() for t in f.readlines() if t.strip()]

    ann_file = f"{ann_base}/participants/train/{doc_id}_AGGREGATED.ann"
    with open(ann_file) as f:
        vals = [int(x) for x in f.read().strip().split(",") if x.strip()]

    print(f"{doc_id}: tokens={len(tokens)}, ann_vals={len(vals)}, match={len(tokens)==len(vals)}")
    print(f"  First 5 tokens: {tokens[:5]}")

10036953: tokens=1, ann_vals=162, match=False
  First 5 tokens: ['[ Triple therapy regimens involving H2 blockaders for therapy of Helicobacter pylori infections ] . Comparison of ranitidine and lansoprazole in short-term low-dose triple therapy for Helicobacter pylori infection . To evaluate the efficacy and safety of two 1-week low-dose triple-therapy drug regimens involving antisecretory drugs for Helicobacter pylori infection , 99 patients with H. pylori infection were treated with either lansoprazole ( LPZ ) or ranitidine ( RNT ) used together with clarithromycin ( CAM ) and metrinidazole ( MTZ ) . The drug combination and administration periods in the PPI group were LPZ 30 mg , CAM 400 mg , MTZ 500 mg ( LCM group ) . The ranitidine group received RNT 300 mg , CAM 400 mg , MTZ 500 mg ( RCM group ) . The cure rate of H. pylori infection was 88 % in the LCM group ; 95 % CI 79-97 and 92 % in the RCM group ; 95 % CI 84-99 .']
10037531: tokens=1, ann_vals=8, match=False
  First 5 token

In [ ]:
import os
import glob
import numpy as np
from sklearn.model_selection import train_test_split

base2 = "EBM-NLP/ebm_nlp_1_00"
ann_base = f"{base2}/annotations/aggregated/starting_spans"
doc_base = f"{base2}/documents"

def load_pico_data_fixed(split="train"):
    label_map = {"participants": "P", "interventions": "I", "outcomes": "O"}

    doc_ids = None
    for entity in label_map:
        split_path = f"{ann_base}/{entity}/{split}"
        if not os.path.exists(split_path):
            return [], []
        ids = set(f.replace("_AGGREGATED.ann", "")
                  for f in os.listdir(split_path)
                  if f.endswith(".ann"))
        doc_ids = ids if doc_ids is None else doc_ids & ids

    all_tokens, all_labels = [], []
    skipped = 0

    for doc_id in sorted(doc_ids):
        tok_file = f"{doc_base}/{doc_id}.tokens"
        if not os.path.exists(tok_file):
            skipped += 1
            continue

        # FIX: read whole file and split on whitespace
        with open(tok_file) as f:
            tokens = f.read().split()

        pico_labels = {}
        valid = True
        for entity, letter in label_map.items():
            ann_file = f"{ann_base}/{entity}/{split}/{doc_id}_AGGREGATED.ann"
            with open(ann_file) as f:
                vals = [int(x) for x in f.read().strip().split(",") if x.strip()]
            if len(vals) != len(tokens):
                skipped += 1
                valid = False
                break
            pico_labels[letter] = vals

        if not valid:
            continue

        labels = []
        for i in range(len(tokens)):
            if   pico_labels["P"][i]: labels.append("B-P")
            elif pico_labels["I"][i]: labels.append("B-I")
            elif pico_labels["O"][i]: labels.append("B-O")
            else:                     labels.append("O")

        all_tokens.append(tokens)
        all_labels.append(labels)

    print(f"Loaded: {len(all_tokens)}, Skipped: {skipped}")
    return all_tokens, all_labels

train_tok, train_lab = load_pico_data_fixed("train")
train_tok, test_tok, train_lab, test_lab = train_test_split(
    train_tok, train_lab, test_size=0.15, random_state=42
)
train_tok, dev_tok, train_lab, dev_lab = train_test_split(
    train_tok, train_lab, test_size=0.1, random_state=42
)

print(f"Train: {len(train_tok)} | Dev: {len(dev_tok)} | Test: {len(test_tok)}")
print(f"Sample tokens: {train_tok[0][:8]}")
print(f"Sample labels: {train_lab[0][:8]}")

Loaded: 4802, Skipped: 0
Train: 3672 | Dev: 409 | Test: 721
Sample tokens: ['Letter', ':', 'Cephaloridine', 'and', 'gentamicin', 'in', 'prophylaxis', 'of']
Sample labels: ['B-P', 'B-P', 'B-P', 'B-P', 'B-P', 'B-P', 'B-P', 'B-P']


full training pipeline:

Dataset class and tokenization

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForTokenClassification, get_linear_schedule_with_warmup
from torch.optim import AdamW
from seqeval.metrics import f1_score, precision_score, recall_score
from tqdm import tqdm

MODEL_NAME = "microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

label2id = {"O": 0, "B-P": 1, "B-I": 2, "B-O": 3}
id2label = {v: k for k, v in label2id.items()}
NUM_LABELS = len(label2id)

class PICODataset(Dataset):
    def __init__(self, all_tokens, all_labels, max_len=512):
        self.samples = []
        for tokens, labels in zip(all_tokens, all_labels):
            enc = tokenizer(
                tokens,
                is_split_into_words=True,
                truncation=True,
                max_length=max_len,
                padding="max_length",
                return_tensors="pt"
            )
            word_ids = enc.word_ids()
            label_ids = []
            prev_word_id = None
            for wid in word_ids:
                if wid is None:
                    label_ids.append(-100)
                elif wid != prev_word_id:
                    label_ids.append(label2id[labels[wid]])
                else:
                    label_ids.append(-100)  # mask subword continuations
                prev_word_id = wid

            self.samples.append({
                "input_ids": enc["input_ids"].squeeze(),
                "attention_mask": enc["attention_mask"].squeeze(),
                "labels": torch.tensor(label_ids)
            })

    def __len__(self): return len(self.samples)
    def __getitem__(self, idx): return self.samples[idx]

print("Building datasets...")
train_dataset = PICODataset(train_tok, train_lab)
dev_dataset   = PICODataset(dev_tok,   dev_lab)
test_dataset  = PICODataset(test_tok,  test_lab)
print(f"Done. Train: {len(train_dataset)}, Dev: {len(dev_dataset)}, Test: {len(test_dataset)}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:949: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Building datasets...
Done. Train: 3672, Dev: 409, Test: 721


In [ ]:
BATCH_SIZE = 16
EPOCHS = 5
LR = 2e-5

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
dev_loader   = DataLoader(dev_dataset,   batch_size=32)
test_loader  = DataLoader(test_dataset,  batch_size=32)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using: {device}")

model = AutoModelForTokenClassification.from_pretrained(
    MODEL_NAME, num_labels=NUM_LABELS,
    id2label=id2label, label2id=label2id
).to(device)

optimizer = AdamW(model.parameters(), lr=LR, weight_decay=0.01)
total_steps = len(train_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(0.1 * total_steps),
    num_training_steps=total_steps
)

def evaluate(loader):
    model.eval()
    all_preds, all_true = [], []
    with torch.no_grad():
        for batch in loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=-1)
            for pred_seq, true_seq in zip(preds, labels):
                p_list, t_list = [], []
                for p, t in zip(pred_seq, true_seq):
                    if t.item() == -100:
                        continue
                    p_list.append(id2label[p.item()])
                    t_list.append(id2label[t.item()])
                all_preds.append(p_list)
                all_true.append(t_list)
    f1 = f1_score(all_true, all_preds)
    p  = precision_score(all_true, all_preds)
    r  = recall_score(all_true, all_preds)
    return f1, p, r

# Training
best_dev_f1 = 0
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    dev_f1, dev_p, dev_r = evaluate(dev_loader)
    print(f"Epoch {epoch+1} | Loss: {avg_loss:.4f} | Dev F1: {dev_f1:.4f} | P: {dev_p:.4f} | R: {dev_r:.4f}")

    if dev_f1 > best_dev_f1:
        best_dev_f1 = dev_f1
        torch.save(model.state_dict(), "best_pico_model.pt")
        print(f"  ✓ New best saved")

# Final test evaluation
model.load_state_dict(torch.load("best_pico_model.pt"))
test_f1, test_p, test_r = evaluate(test_loader)
print(f"\nTest F1: {test_f1:.4f} | P: {test_p:.4f} | R: {test_r:.4f}")
print(f"Paper target: 73.38")

Using: cuda


Some weights of BertForTokenClassification were not initialized from the model checkpoint at microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1: 100%|██████████| 230/230 [06:02<00:00,  1.58s/it]


Epoch 1 | Loss: 0.6735 | Dev F1: 0.7239 | P: 0.7546 | R: 0.6956
  ✓ New best saved


Epoch 2: 100%|██████████| 230/230 [06:03<00:00,  1.58s/it]


Epoch 2 | Loss: 0.4117 | Dev F1: 0.7335 | P: 0.7406 | R: 0.7265
  ✓ New best saved


...

Epoch 3 | Loss: 0.3764 | Dev F1: 0.7252 | P: 0.7567 | R: 0.6962


Epoch 4: 100%|██████████| 230/230 [06:03<00:00,  1.58s/it]


Epoch 4 | Loss: 0.3514 | Dev F1: 0.7328 | P: 0.7449 | R: 0.7211


Epoch 5: 100%|██████████| 230/230 [06:02<00:00,  1.58s/it]


Epoch 5 | Loss: 0.3331 | Dev F1: 0.7315 | P: 0.7584 | R: 0.7064

Test F1: 0.7271 | P: 0.7341 | R: 0.7202
Paper target: 73.38
